# 속도 제한 처리 방법

OpenAI API를 반복적으로 호출하면 `429: '요청이 너무 많습니다'` 또는 `RateLimitError`라는 오류 메시지가 표시될 수 있습니다. 이러한 오류 메시지는 API의 속도 제한을 초과하여 발생합니다.

이 가이드에서는 전송률 제한 오류를 방지하고 처리하기 위한 팁을 공유합니다.

속도 제한 오류를 피하기 위해 병렬 요청을 스로틀링하는 예제 스크립트를 보려면 [api_request_parallel_processor.py](api_request_parallel_processor.py)를 참조하세요.

------

When you call the OpenAI API repeatedly, you may encounter error messages that say `429: 'Too Many Requests'` or `RateLimitError`. These error messages come from exceeding the API's rate limits.

This guide shares tips for avoiding and handling rate limit errors.

To see an example script for throttling parallel requests to avoid rate limit errors, see [api_request_parallel_processor.py](api_request_parallel_processor.py).

## 속도 제한이 존재하는 이유

속도 제한은 API의 일반적인 관행이며, 몇 가지 다른 이유로 설정됩니다.

- 첫째, API의 남용이나 오용을 방지하는 데 도움이 됩니다. 예를 들어, 악의적인 공격자가 API에 과부하를 일으키거나 서비스 중단을 유발하기 위해 요청을 폭주시킬 수 있습니다. 속도 제한을 설정하면 이러한 종류의 활동을 방지할 수 있습니다.
- 둘째, 속도 제한은 모든 사람이 API에 공정하게 액세스할 수 있도록 보장합니다. 한 사람이나 조직이 지나치게 많은 요청을 하면 다른 모든 사람의 API가 느려질 수 있습니다. OpenAI는 한 사용자가 요청할 수 있는 요청 수를 제한함으로써 모든 사용자가 속도 저하 없이 API를 사용할 수 있도록 보장합니다.
- 마지막으로, 속도 제한은 OpenAI가 인프라의 총 부하를 관리하는 데 도움이 될 수 있습니다. API에 대한 요청이 급격히 증가하면 서버에 부담을 주고 성능 문제를 일으킬 수 있습니다. 속도 제한을 설정하면 OpenAI는 모든 사용자에게 원활하고 일관된 환경을 유지할 수 있습니다.

속도 제한에 도달하는 것은 실망스러울 수 있지만, 속도 제한은 사용자를 위한 API의 안정적인 운영을 보호하기 위해 존재합니다.

---------------

Rate limits are a common practice for APIs, and they're put in place for a few different reasons.

- First, they help protect against abuse or misuse of the API. For example, a malicious actor could flood the API with requests in an attempt to overload it or cause disruptions in service. By setting rate limits, OpenAI can prevent this kind of activity.
- Second, rate limits help ensure that everyone has fair access to the API. If one person or organization makes an excessive number of requests, it could bog down the API for everyone else. By throttling the number of requests that a single user can make, OpenAI ensures that everyone has an opportunity to use the API without experiencing slowdowns.
- Lastly, rate limits can help OpenAI manage the aggregate load on its infrastructure. If requests to the API increase dramatically, it could tax the servers and cause performance issues. By setting rate limits, OpenAI can help maintain a smooth and consistent experience for all users.

Although hitting rate limits can be frustrating, rate limits exist to protect the reliable operation of the API for its users.

## Default rate limits

As of Jan 2023, the default rate limits are:

<table>
<thead>
  <tr>
    <th></th>
    <th>Text Completion &amp; Embedding endpoints</th>
    <th>Code &amp; Edit endpoints</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Free trial users</td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Pay-as-you-go users (in your first 48 hours)</td>
    <td>
        <ul>
            <li>60 requests / minute</li>
            <li>250,000 davinci tokens / minute (and proportionally more for cheaper models)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Pay-as-you-go users (after your first 48 hours)</td>
    <td>
        <ul>
            <li>3,000 requests / minute</li>
            <li>250,000 davinci tokens / minute (and proportionally more for cheaper models)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 requests / minute</li>
            <li>150,000 tokens / minute</li>
        </ul>
    </td>
  </tr>
</tbody>
</table>

참고로 1,000토큰은 대략 텍스트 한 페이지에 해당합니다.

--------

For reference, 1,000 tokens is roughly a page of text.

### 기타 속도 제한 리소스

다른 리소스에서 OpenAI의 전송률 제한에 대해 자세히 알아보세요:

- 가이드: 비율 제한](https://beta.openai.com/docs/guides/rate-limits/overview)
- 도움말 센터: API 사용에 요금 제한이 적용되나요?(https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits)
- 도움말 센터: 429: '너무 많은 요청' 오류는 어떻게 해결하나요?](https://help.openai.com/en/articles/5955604-how-can-i-solve-429-too-many-requests-errors)

------------------

Read more about OpenAI's rate limits in these other resources:

- [Guide: Rate limits](https://beta.openai.com/docs/guides/rate-limits/overview)
- [Help Center: Is API usage subject to any rate limits?](https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits)
- [Help Center: How can I solve 429: 'Too Many Requests' errors?](https://help.openai.com/en/articles/5955604-how-can-i-solve-429-too-many-requests-errors)

### 비율 제한 증가 요청하기

조직의 요금 한도를 늘리려면 다음 양식을 작성해 주세요:

- OpenAI 요금 한도 증액 요청 양식](https://forms.gle/56ZrwXXoxAN1yt6i9)

---------------------

If you'd like your organization's rate limit increased, please fill out the following form:

- [OpenAI Rate Limit Increase Request form](https://forms.gle/56ZrwXXoxAN1yt6i9)


## 전송률 제한 오류 예시

API 요청이 너무 빨리 전송되면 속도 제한 오류가 발생합니다. OpenAI Python 라이브러리를 사용하는 경우 다음과 같이 표시됩니다:

```
RateLimitError: 조직 org-{id}의 기본 코덱스에 대한 분당 요청 수에 대한 속도 제한에 도달했습니다. 제한: 20.000000/분. 현재: 24.000000/분. 문제가 계속되거나 증가를 요청하려면 support@openai.com 으로 문의하세요.
```

다음은 속도 제한 오류를 트리거하는 예제 코드입니다.

-----

A rate limit error will occur when API requests are sent too quickly. If using the OpenAI Python library, they will look something like:

```
RateLimitError: Rate limit reached for default-codex in organization org-{id} on requests per min. Limit: 20.000000 / min. Current: 24.000000 / min. Contact support@openai.com if you continue to have issues or if you’d like to request an increase.
```

Below is example code for triggering a rate limit error.

In [1]:
import openai  # for making OpenAI API requests

# request a bunch of completions in a loop
for _ in range(100):
    openai.Completion.create(
        model="code-cushman-001",
        prompt="def magic_function():\n\t",
        max_tokens=10,
    )


RateLimitError: Rate limit reached for default-codex in organization org-dLiMqqSmLIXuV45cMCo7Pvcj on requests per min. Limit: 20 / min. Please try again in 3s. Contact support@openai.com if you continue to have issues or if you’d like to request an increase.

## 속도 제한 오류를 방지하는 방법

### 지수 백오프를 사용하여 재시도하기

비율 제한 오류를 피하는 쉬운 방법 중 하나는 무작위 지수 백오프를 사용하여 요청을 자동으로 재시도하는 것입니다. 지수 백오프를 사용하여 재시도한다는 것은 비율 제한 오류가 발생하면 짧은 절전을 수행한 다음 실패한 요청을 다시 시도하는 것을 의미합니다. 요청이 여전히 실패하면 절전 시간이 길어지고 프로세스가 반복됩니다. 이 과정은 요청이 성공하거나 최대 재시도 횟수에 도달할 때까지 계속됩니다.

이 접근 방식에는 많은 이점이 있습니다:

- 자동 재시도는 충돌이나 데이터 누락 없이 속도 제한 오류에서 복구할 수 있음을 의미합니다.
- 지수 백오프를 사용하면 첫 번째 재시도를 빠르게 시도할 수 있으며, 처음 몇 번의 재시도가 실패할 경우 지연 시간이 길어지는 이점을 누릴 수 있습니다.
- 지연에 무작위 지터를 추가하면 모든 재시도를 동시에 시도하는 데 도움이 됩니다.

실패한 요청은 분당 요청 횟수 제한에 영향을 미치므로 요청을 계속 재전송하는 것은 효과가 없습니다.

다음은 몇 가지 해결 방법의 예입니다.

-----------

One easy way to avoid rate limit errors is to automatically retry requests with a random exponential backoff. Retrying with exponential backoff means performing a short sleep when a rate limit error is hit, then retrying the unsuccessful request. If the request is still unsuccessful, the sleep length is increased and the process is repeated. This continues until the request is successful or until a maximum number of retries is reached.

This approach has many benefits:

- Automatic retries means you can recover from rate limit errors without crashes or missing data
- Exponential backoff means that your first retries can be tried quickly, while still benefiting from longer delays if your first few retries fail
- Adding random jitter to the delay helps retries from all hitting at the same time

Note that unsuccessful requests contribute to your per-minute limit, so continuously resending a request won’t work.

Below are a few example solutions.

#### 예제 #1: Tenacity 라이브러리 사용

[Tenacity](https://tenacity.readthedocs.io/en/latest/)는 거의 모든 것에 재시도 동작을 추가하는 작업을 단순화하기 위해 Python으로 작성된 Apache 2.0 라이센스가 있는 범용 재시도 라이브러리입니다.

요청에 기하급수적 백오프를 추가하려면 `tenacity.retry` [decorator](https://peps.python.org/pep-0318/)를 사용하면 됩니다. 다음 예제는 `tenacity.wait_random_exponential` 함수를 사용하여 요청에 무작위 지수 백오프를 추가하는 예제입니다.

Tenacity 라이브러리는 타사 도구이며, OpenAI는 안정성이나 보안에 대해 보장하지 않는다는 점에 유의하세요.

--------------------

[Tenacity](https://tenacity.readthedocs.io/en/latest/) is an Apache 2.0 licensed general-purpose retrying library, written in Python, to simplify the task of adding retry behavior to just about anything.

To add exponential backoff to your requests, you can use the `tenacity.retry` [decorator](https://peps.python.org/pep-0318/). The following example uses the `tenacity.wait_random_exponential` function to add random exponential backoff to a request.

Note that the Tenacity library is a third-party tool, and OpenAI makes no guarantees about its reliability or security.

In [2]:
import openai  # for OpenAI API calls
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completion_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-6tk5HgLHXQbj5JQMdeI8B9iEZFzYo at 0x12027e8e0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " I did a dumb thing.\n\nI was at school and I saw a"
    }
  ],
  "created": 1678743207,
  "id": "cmpl-6tk5HgLHXQbj5JQMdeI8B9iEZFzYo",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예제 #2: 백오프 라이브러리 사용

백오프 및 재시도를 위한 함수 데코레이터를 제공하는 또 다른 라이브러리는 [backoff](https://pypi.org/project/backoff/)입니다.

Tenacity와 마찬가지로 백오프 라이브러리는 타사 도구이며, OpenAI는 안정성이나 보안에 대해 보장하지 않습니다.

-------------

Another library that provides function decorators for backoff and retry is [backoff](https://pypi.org/project/backoff/).

Like Tenacity, the backoff library is a third-party tool, and OpenAI makes no guarantees about its reliability or security.

In [3]:
import backoff  # for exponential backoff
import openai  # for OpenAI API calls


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-6tkBFXNFwJYSGuVc3Jrlsj1lUbqcY at 0x10817ca90> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " on a sunny but cold winter Wednesday, a southbound Amtrak train rolled out of"
    }
  ],
  "created": 1678743577,
  "id": "cmpl-6tkBFXNFwJYSGuVc3Jrlsj1lUbqcY",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예제 3: 수동 백오프 구현

타사 라이브러리를 사용하고 싶지 않다면 자체 백오프 로직을 구현할 수 있습니다.

------------

If you don't want to use third-party libraries, you can implement your own backoff logic.

In [4]:
# imports
import random
import time

import openai

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    errors: tuple = (openai.error.RateLimitError,),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specified errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper


@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-6tkCa6xbwqRLoma35tz8NzMP4EySA at 0x120148360> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " there was anKing known as Henry Kenedy who was the best knitter"
    }
  ],
  "created": 1678743660,
  "id": "cmpl-6tkCa6xbwqRLoma35tz8NzMP4EySA",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

## 속도 제한이 주어졌을 때 일괄 처리 처리량을 최대화하는 방법

사용자의 실시간 요청을 처리하는 경우, 백오프 및 재시도는 속도 제한 오류를 방지하면서 지연 시간을 최소화하는 훌륭한 전략입니다.

하지만 지연 시간보다 처리량이 더 중요한 대량의 배치 데이터를 처리하는 경우에는 백오프 및 재시도 외에 몇 가지 다른 방법을 사용할 수 있습니다.

-------

If you're processing real-time requests from users, backoff and retry is a great strategy to minimize latency while avoiding rate limit errors.

However, if you're processing large volumes of batch data, where throughput matters more than latency, there are a few other things you can do in addition to backoff and retry.

### 선제적으로 요청 사이에 지연 시간 추가하기

속도 제한에 도달했다가 백오프하고, 다시 속도 제한에 도달하고, 다시 백오프하는 과정을 반복하면 요청 예산의 상당 부분이 재시도해야 하는 요청에 '낭비'될 수 있습니다. 이렇게 되면 고정된 비율 제한이 주어지면 처리량이 제한됩니다.

여기서 한 가지 잠재적인 해결책은 속도 제한을 계산하고 그 역수와 같은 지연을 추가하는 것입니다(예: 속도 제한이 분당 요청 20건인 경우 각 요청에 3~6초의 지연을 추가). 이렇게 하면 한도에 도달하여 요청이 낭비되지 않고 요금 한도 근처에서 운영할 수 있습니다.

---------

If you are constantly hitting the rate limit, then backing off, then hitting the rate limit again, then backing off again, it's possible that a good fraction of your request budget will be 'wasted' on requests that need to be retried. This limits your processing throughput, given a fixed rate limit.

Here, one potential solution is to calculate your rate limit and add a delay equal to its reciprocal (e.g., if your rate limit 20 requests per minute, add a delay of 3–6 seconds to each request). This can help you operate near the rate limit ceiling without hitting it and incurring wasted requests.

#### 요청에 지연을 추가하는 예

In [5]:
# imports
import time
import openai

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return openai.Completion.create(**kwargs)


# Calculate the delay based on your rate limit
rate_limit_per_minute = 20
delay = 60.0 / rate_limit_per_minute

delayed_completion(
    delay_in_seconds=delay,
    model="text-davinci-002",
    prompt="Once upon a time,"
)


<OpenAIObject text_completion id=cmpl-6tkDlNKG5rCmaRlh7AmEvleikj0Cy at 0x108106520> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " Mr. Kumar brought radishes from the market.\n\nHe was very happy"
    }
  ],
  "created": 1678743733,
  "id": "cmpl-6tkDlNKG5rCmaRlh7AmEvleikj0Cy",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

### 요청 일괄 처리

OpenAI API에는 분당 요청 수와 분당 토큰 수에 대한 별도의 제한이 있습니다.

분당 요청 수 제한에 도달했지만 분당 토큰 수에 여유가 있는 경우, 여러 작업을 각 요청에 일괄 처리하여 처리량을 늘릴 수 있습니다. 이렇게 하면 특히 소규모 모델에서 분당 더 많은 토큰을 처리할 수 있습니다.

프롬프트 일괄 전송은 일반 API 호출과 완전히 동일하게 작동하지만, 단일 문자열 대신 '프롬프트' 매개변수에 문자열 목록을 전달한다는 점이 다릅니다.

**경고: 응답 객체가 프롬프트 순서대로 완료를 반환하지 않을 수 있으므로 항상 `index` 필드를 사용하여 응답을 프롬프트에 다시 일치시켜야 합니다.

--------

The OpenAI API has separate limits for requests per minute and tokens per minute.

If you're hitting the limit on requests per minute, but have headroom on tokens per minute, you can increase your throughput by batching multiple tasks into each request. This will allow you to process more tokens per minute, especially with the smaller models.

Sending in a batch of prompts works exactly the same as a normal API call, except that pass in a list of strings to `prompt` parameter instead of a single string.

**Warning:** the response object may not return completions in the order of the prompts, so always remember to match responses back to prompts using the `index` field.

#### 일괄 처리하지 않은 예제

In [11]:
import openai  # for making OpenAI API requests


num_stories = 10
prompt = "Once upon a time,"
prompt = "옛날 옛적에,"

# serial example, with one story completion per request
for _ in range(num_stories):
    response = openai.Completion.create(
        model="text-davinci-003",
#         model="davinci",
#         model="curie",
        prompt=prompt,
        max_tokens=200,
    )

    # print story
    print(prompt + response.choices[0].text)


옛날 옛적에, 숙녀가 머리 빗물
옛날 옛적에,
우리 마을에는

옛날 옛적에, 동네에는 우리할
옛날 옛적에, 젊은이들이 가릴
옛날 옛적에,

옛날 옛적에는
옛날 옛적에,

교회가 중요했
옛날 옛적에, 한 나라 곳곳을
옛날 옛적에, 고등학교 학생
옛날 옛적에, 장군은 기습을 
옛날 옛적에, 기사들은 매우 열
옛날 옛적에, 모니터는 대부분
옛날 옛적에, 특히 일제나 다
옛날 옛적에, 궁궐 안에는 총
옛날 옛적에, 길이 멀리서는
옛날 옛적에,

옛날 옛적에는
옛날 옛적에, 한 나라 속 금빛 
옛날 옛적에, 훗날 옛적에는 
옛날 옛적에, 나는 작은 골목
옛날 옛적에, 고기를 씻는 방
옛날 옛적에, 금빛 빵 가게가
옛날 옛적에, 사람들이

옛날
옛날 옛적에, 개미가 개구리
옛날 옛적에, 고향 속 명물인 뿔
옛날 옛적에,

옛날 옛날에는
옛날 옛적에, 고대 명목 개념
옛날 옛적에, '가깝길'은 없다
옛날 옛적에, 그리운 추억들

옛날 옛적에, 이상한 이야기가
옛날 옛적에,
큰가지 길이 있
옛날 옛적에, 아마존 우리가
옛날 옛적에, 나의 이름을 불러
옛날 옛적에, 내 손끝 잡고 갔
옛날 옛적에, 생활 그리고 사
옛날 옛적에,
사람들은 멍멍이
옛날 옛적에, 그들 사이에 한 남
옛날 옛적에, 한 산골에 가장
옛날 옛적에, 한 농부가 옆밭
옛날 옛적에,
길 가는 난민에
옛날 옛적에,
>
> 스스로 혼자
옛날 옛적에, 방금, 애매

**옛
옛날 옛적에, 5살 어린 엔딩이
옛날 옛적에, 먼지가 많은 창문
옛날 옛적에, 말로는 재배가
옛날 옛적에, 가난한 일꾼이 무
옛날 옛적에,

고대 시대 때에
옛날 옛적에, 나는 학교에 갔
옛날 옛적에,

옛날 옛적에는
옛날 옛적에, 그 밤하늘에 떠
옛날 옛적에, 아이 오 드 테이 
옛날 옛적에, 내 늙장이가 다음
옛날 옛적에, 아들자식이 손님
옛날 옛적에,
  어느 승려가 나
옛날 옛적에, 사람들은

집에
옛날 옛적에, 산다는 말은
어떤
옛날 옛적에,

나라 가는 노
옛날 옛적에, 모두 이야기 하
옛날 옛적에, 옛날 사람들은 이
옛날 옛적에, 대

#### Example with batching

In [19]:
import openai  # for making OpenAI API requests


num_stories = 10
# prompts = ["Once upon a time,"] * num_stories
prompts = ["옛날 옛적에,"] * num_stories

# batched example, with 10 stories completions per request
response = openai.Completion.create(
    model="text-davinci-003",
#     model="davinci",
#     model="curie",
    prompt=prompts,
    max_tokens=200,
)

# match completions to prompts by index
stories = [""] * len(prompts)
for choice in response.choices:
    stories[choice.index] = prompts[choice.index] + choice.text

# print stories
for story in stories:
    print(story)
    print("---------------------")


옛날 옛적에, 옛날 옛적에 아버지가 목장에 양 한 마리를 가지고 갔어요. 그는 매일 양과 함께 저녁 때 먹을 채소를 심어 놓았습니다. 이 채소는 일년 내내 마른 계절이나 비오는 겨울에
---------------------
옛날 옛적에, 용사가 산 위로 올라갔다. 한 병기를 차고 노래를 불러도록 하길 바랬던 백성들이 어느새 군량에 모이기 시작했다. 용사 앞에 무리가 펼쳐지자 그의 눈과 가슴 모두 무척 충
---------------------
옛날 옛적에, 빨간 소녀 이렇게 말했다 

"오늘도 에세이를 써봐야겠다. 내가 무슨 생각이 있든, 내가 무엇을 놀라게 할 수 있든, 이 글에서 녹아 내는 감정이 누구를 생각하게 할까? 그것
---------------------
옛날 옛적에, 쏘레(호박 라디오)에 호피가 게임을 개발하였다. 호피가 개발한 게임은 원작으로써 밀다르 게임이라는 이름의 동물 가족을 연결하는 게임을 의미하였다. 이 게임은 다른
---------------------
옛날 옛적에, 사람들은 천주교가 전해주는 믿음이 최고의 진리라고 믿었습니다. 그들은 그 내용을 신중하게 추론하고 생각하였으며, 이것을 지킬 것이라 생각하였습니다. 그렇게 그들은 사람
---------------------
옛날 옛적에, 우리 떤 남자 였다

옛날 옛적에 우리는 다섯 남자가 있었다. 그 남자들은 힘찬 용맹과 의지로 자신들의 삶을 이뤄내기 위해 노력하는 독특한 문화가 있었고, 그들의 삶에는
---------------------
옛날 옛적에, <strong>못 먹게 만든 배알이 길들인다</strong>라는 속담이 있습니다.
이 속담은 상황이 어려워졌을 때를 의미하며, 하나의 노력이라도 하면 나아지게 된다는 뜻을 이끌어 냅니다. 이 속담에 따
---------------------
옛날 옛적에, 한 산 꼭대기에 이성 깊이
우리 둘이 다툼이었죠
별들 소리도 나오고 바람도 불었죠
그런 밤이 오면 우리는 잠들었죠

길 건너 사람들은 모두 잊혀지고 버려지고
나만
----------

## 병렬 처리 스크립트 예시

대량의 API 요청을 병렬 처리하기 위한 예제 스크립트를 작성했습니다: [api_request_parallel_processor.py](api_request_parallel_processor.py).

이 스크립트에는 몇 가지 편리한 기능이 결합되어 있습니다:
- 대규모 작업으로 인한 메모리 부족을 방지하기 위해 파일에서 요청을 스트리밍합니다.
- 처리량을 최대화하기 위해 요청을 동시에 수행합니다.
- 요청과 토큰 사용량을 모두 스로틀하여 속도 제한을 준수합니다.
- 데이터 누락을 방지하기 위해 실패한 요청을 재시도합니다.
- 오류를 기록하여 요청 문제를 진단합니다.

있는 그대로 사용하거나 필요에 맞게 자유롭게 수정하세요.

---------------------

We've written an example script for parallel processing large quantities of API requests: [api_request_parallel_processor.py](api_request_parallel_processor.py).

The script combines some handy features:
- Streams requests from file, to avoid running out of memory for giant jobs
- Makes requests concurrently, to maximize throughput
- Throttles both request and token usage, to stay under rate limits
- Retries failed requests, to avoid missing data
- Logs errors, to diagnose problems with requests

Feel free to use it as is or modify it to suit your needs.